In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import warnings

sys.path.append('../src/')
warnings.filterwarnings("ignore")

from server import Server
from utils import plot_exp

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [6, 6]
plt.rcParams['figure.dpi'] = 100

### First experiment : impact of federated learning

In [4]:
LR = 0.01
EPOCHS = 6
NR_TRAINING_ROUNDS = 3
BATCH_SIZE = 128

RANGE_NR_CLIENTS = [1,5,10]

In [5]:
experiment_losses, experiment_accs = [], []
for nr_clients in RANGE_NR_CLIENTS:
    print(f"### Number of clients : {nr_clients} ###\n\n")
    server = Server(
        nr_clients=nr_clients,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='FEMNIST',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=False,
        epsilon=None,
        max_grad_norm=None,
        noise_multiplier=None,
        is_parallel=True,
        device=None,
        verbose='all')

    test_losses, test_accs = server.train(early=False, patience=2)
    
    experiment_losses.append(test_losses)
    experiment_accs.append(test_accs)

### Number of clients : 1 ###


--- Configuration ---
nr_clients: 1
nr_training_rounds: 3
data: FEMNIST
epochs: 6
lr: 0.01
batch_size: 128
is_private: False
epsilon: None
max_grad_norm: None
noise_multiplier: None
is_parallel: True
device: <class 'torch.device'>
verbose: all
--- Loading Data ---


FileNotFoundError: [Errno 2] File ../data/FEMNIST/test.csv does not exist: '../data/FEMNIST/test.csv'

In [ ]:
names = [f'{i} clients' for i in RANGE_NR_CLIENTS]
plot_exp(experiment_losses, experiment_accs, names)

### Second experiment : impact of differential privacy

In [ ]:
NR_CLIENTS = 3
NR_TRAINING_ROUNDS = 6
EPOCHS = 6
PATIENCE = 4
LR = 0.01
BATCH_SIZE = 128

MAX_GRAD_NORM = 1.2
NOISE_MULTIPLIER = None

RANGE_EPSILON = [60]

In [ ]:
experiment_losses, experiment_accs = [], []
for epsilon in RANGE_EPSILON:
    print(f"### epsilon : {epsilon} ###\n\n")
    server = Server(
        nr_clients=NR_CLIENTS,
        nr_training_rounds=NR_TRAINING_ROUNDS,
        data='MNIST',
        epochs=EPOCHS,
        lr=LR,
        batch_size=BATCH_SIZE,
        is_private=True,
        epsilon=epsilon,
        max_grad_norm=MAX_GRAD_NORM,
        noise_multiplier=NOISE_MULTIPLIER,
        is_parallel=True,
        device=None,
        verbose='all')

    test_losses, test_accs = server.train(early=False, patience=PATIENCE)
    experiment_losses.append(test_losses)
    experiment_accs.append(test_accs)

In [ ]:
names = [f'ε = {i}' for i in RANGE_EPSILON]
plot_exp(experiment_losses, experiment_accs, names)